In [1]:
data1 = [
    (1,"a1","BSAS"),
    (2,"a2","BSAS"),
    (3,"a3","BSAS"),
    (4,"a4","Noruega")
]

data2 = [
    ("2018-08-12", 1, 30),
    ("2018-08-14", 4, 30),
    ("2018-07-13", 1, 20),
    ("2018-08-14", 1, 30),
    ("2018-08-14", 1, 30),
    ("2017-07-12", 1, 30),
    ("2018-08-12", 1, 30),
    ("2018-08-12", 1, 30),
]

In [2]:
#rdd1 - id_base , nombre , pcia , ciudad , lat , lon
rdd1 = sc.parallelize(data1);

#rdd2 - TIMESTAMP, ID_BASE, TEMPERATURA, HUMEDAD, 
#       PRESIÓN, DIRECCIÓN VIENTO, VELOCIDAD VIENTO
rdd2 = sc.parallelize(data2);

#ID y nombre result final

In [3]:
rdd1 = rdd1.filter(lambda x: x[2]=="BSAS")
rdd2 = rdd2.filter(lambda x: x[0]>="2017-12-00")

In [4]:
rdd1 = rdd1.map(lambda x: (x[0],x[1]))
#rdd1 -> id_base, nombre

rdd2 = rdd2.map(lambda x: ((x[1],x[0][:7]),(x[2],1) ) )
#rdd2 -> (año-mes, id_base) , (temp, 1)

In [5]:
rdd2 = rdd2.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
#rdd2 -> (año-mes, id_base) , (suma de temp, cant de mediciones)

In [6]:
def mes_anterior(x):
    mes=int(x[5:7])
    anio=int(x[:4])
    mes=mes-1
    if mes==0:
        anio=anio-1
        mes=12
    return (str(anio)+"-"+"0"+str(mes))


In [7]:
rdd2 = rdd2.flatMap(lambda x: [  ( ( x[0][0],mes_anterior(x[0][1]) ), x[1][0]/x[1][1] ),
                        ( ( x[0][0],x[0][1] ), x[1][0]/x[1][1] ) ] )
#rdd2 -> id_base , (promedio, anio mes)


In [8]:
rdd2 = rdd2.groupByKey().mapValues(list)
# el map values es solo para ver los valores que tengo, sino no puedo verlo desde aca

# id_base, ((prom, mes),(prom,mes_anterior))
rdd2.collect()

[((1, '2018-06'), [20.0]),
 ((1, '2018-07'), [30.0, 20.0]),
 ((1, '2018-08'), [30.0]),
 ((4, '2018-07'), [30.0]),
 ((4, '2018-08'), [30.0])]

In [9]:
#Filtro los del 30% 
rdd2 = rdd2.filter(lambda x: (len(x[1])==2) and ((x[1][1]/x[1][0]) < 0.70 ))

#Mapeo para joinear
rdd2 = rdd2.map(lambda x: (x[0][0],x[0][1]))
#rdd2 --> id_base, mes

In [10]:
result = rdd1.join(rdd2)
result.map(lambda x: (x[0],x[1][0])).collect()

[(1, 'a1')]